In [1]:
from vidCon.vcon_class import InitiateVcon
from vidCon.videoCon import GetVcon
from vidCon.NER import PrivacyFilter
from vidCon.SoundScrubber import SoundScrubber

/Users/suraj/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
vCon = InitiateVcon()

In [3]:
#initialize vcon
audio_file_name = "./uploads/Data_Dr.m4a"

vcon_processor = GetVcon(audio_file_name)
vcon_processor.read_audio()
vcon_processor.decode_audio()
vcon_processor.writeTempFile()
vcon_processor.transcribe_audio()

Audio read complete
Transcription
Transcription complete


In [4]:
transcribed_text = vcon_processor.vcon_['transcription']['transcribed_text']
transcribed_timestamps = vcon_processor.vcon_['transcription']['transcribed_word_timestamps']

In [5]:
privacy_filter = PrivacyFilter()
entities = privacy_filter.getEntities(transcribed_text)

--- Model loading.... ---
--- Model loaded! ---


/Users/suraj/Library/Python/3.8/lib/python/site-packages/transformers/pipelines/token_classification.py:392: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


In [6]:
entities

[{'entity_group': 'FIRSTNAME',
  'score': 0.7694283,
  'word': ' John',
  'start': 14,
  'end': 19},
 {'entity_group': 'FIRSTNAME',
  'score': 0.40592912,
  'word': ' John',
  'start': 39,
  'end': 44},
 {'entity_group': 'CREDITCARDISSUER',
  'score': 0.6680575,
  'word': ' example.com.',
  'start': 51,
  'end': 64},
 {'entity_group': 'STREETADDRESS',
  'score': 0.8340771,
  'word': ' 1234 Main Street Springfield,',
  'start': 74,
  'end': 104},
 {'entity_group': 'STATE',
  'score': 0.9968298,
  'word': ' Illinois.',
  'start': 104,
  'end': 114},
 {'entity_group': 'ZIPCODE',
  'score': 0.554984,
  'word': ' 555',
  'start': 133,
  'end': 137},
 {'entity_group': 'PHONE_NUMBER',
  'score': 0.4983812,
  'word': ' 1234567.',
  'start': 137,
  'end': 146}]

In [7]:
redactions = sorted(entities, key=lambda x: x['start'], reverse=True)
t = transcribed_text
for redaction in redactions:
    start = redaction['start']
    end = redaction['end']
    # Replace the specified range with '<Redacted>'
    t = t[:start] + '<Redacted>' + t[end:]
t

' Hi my name is<Redacted> Doe and my email is<Redacted> Doe at<Redacted> I live in<Redacted><Redacted> My phone number is<Redacted><Redacted>'

In [8]:
sound_scrubber = SoundScrubber(audio_file_name)
words_to_silence = sound_scrubber.getWordsToSilence(entities)
words_to_silence  = [j for i in words_to_silence for j in i]
sound_scrubber.silenceAudio(words_to_silence, transcribed_timestamps)
print("Processing complete. Check the output audio file for redacted content.")

Redacted audio saved at ./uploads/redacted.mp3
Processing complete. Check the output audio file for redacted content.


In [16]:
transcribed_timestamps[20:]

[{'word': ' Street',
  'start': 8.0,
  'end': 8.48,
  'probability': 0.9401575922966003},
 {'word': ' Springfield,',
  'start': 8.48,
  'end': 9.36,
  'probability': 0.8483898043632507},
 {'word': ' Illinois.',
  'start': 9.54,
  'end': 9.94,
  'probability': 0.9454324841499329}]

In [9]:
from datetime import datetime

In [10]:
dt = datetime.now()
normal_form = dt.strftime("%Y-%m-%d %H:%M:%S.%f")
normal_form

'2024-03-24 02:11:08.002852'

In [11]:
vCon.json_data['dialog'][0]['timestamp'] = normal_form

In [27]:
vCon.json_data['dialog'][0]['content'] = vcon_processor.encoded_bytes

In [28]:
vCon.json_data['analysis']['transcriptions'][0]['text'] = transcribed_text

In [29]:
vCon.json_data['analysis']['entities'] = entities

In [30]:
vCon.json_data['analysis']['transcribed_timestamps'] = transcribed_timestamps